# DerivKit — Fisher Information Demo

This notebook shows how to use **ForecastKit** to compute the Fisher information matrix for parameters 
$\theta = (\theta_1, \theta_2)$ given model-predicted **observables** $o = (o_1, o_2)$.

### What are parameters and observables?
- **Parameters (theta):** the unknown quantities we want to estimate; here these are $\theta_1$ and $\theta_2$.
- **Observables:** the model outputs we would measure; here these are $o_1$ and $o_2$.

The Fisher matrix tells us how precisely we can determine the **parameters** from the **observables** (given a data covariance).

#### Model and Jacobian
Model mapping parameters to observables:

$$
o_1 = \theta_1 + \theta_2, \qquad
o_2 = \theta_1^2 + 2\,\theta_2^2
$$

Jacobian (how observables change with parameters):

$$
J(\theta_1, \theta_2) = \begin{bmatrix}
1 & 1 \\
2\,\theta_1 & 4\,\theta_2
\end{bmatrix}
$$

### Notes:

Assuming the data covariance matrix is the identity, the Fisher matrix is
computed as J-transpose times J. The inverse of the Fisher matrix gives the
parameter covariance, from which we can read off the 1-sigma and 2-sigma
uncertainties on θ1 and θ2.
In other words, the observables are what the model predicts, and the Fisher
matrix quantifies how precisely we can estimate the parameters that produce
those observables.

- If ``method`` is omitted, the **adaptive** backend is used.
- You can force a backend via ``method="finite"`` (or another registered one).
- To list available methods at runtime, use
  ``from derivkit.derivative_kit import available_methods``.

### Requirements
- ``derivkit`` importable in your environment.
- ``numpy`` for numerical computations.
- ``matplotlib`` for plotting (if --plot is used).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from derivkit import ForecastKit

# Pretty printing
np.set_printoptions(precision=6, suppress=True)

# Style tweaks per preferences: larger font, no grids, tight layout by default
plt.rcParams.update({
    'figure.figsize': (6.0, 5.0),
    'axes.titlesize': 14,
    'axes.labelsize': 13,
    'legend.fontsize': 12,
})

# Confidence levels
PCT_1D_1SIG = 0.682689492  # 68.27%
PCT_1D_2SIG = 0.954499736  # 95.45%
PCT_2D_1SIG = 0.683  # 68.3%
PCT_2D_2SIG = 0.955  # 95.5%

In [ ]:
# Model, Jacobian, and analytic Fisher
def model(theta):
    """Map parameters (theta1, theta2) to observables (o1, o2)."""
    theta1 = float(theta[0])
    theta2 = float(theta[1])
    o1 = theta1 + theta2
    o2 = theta1*theta1 + 2.0*theta2*theta2
    return np.array([o1, o2], dtype=float)

def jacobian_analytic(theta1: float, theta2: float):
    """Analytic Jacobian of `my_function`.

    Args:
        theta1: Parameter theta1.
        theta2: Parameter theta2.

    Returns:
        Jacobian matrix J (shape 2x2).
    """
    return np.array([[1.0, 1.0],
                     [2.0*theta1, 4.0*theta2]], dtype=float)

def fisher_analytic(theta1: float, theta2: float, cov: np.ndarray) -> np.ndarray:
    """Analytic Fisher matrix for the model at (x, y) with data covariance `cov`.

    Args:
        theta1: Parameter theta1.
        theta2: Parameter theta2.
        cov: Data covariance matrix (shape 2x2).

    Returns:
        Fisher information matrix (shape 2x2).
    """
    jac = jacobian_analytic(theta1, theta2)
    cov_inv = np.linalg.inv(cov)
    return jac.T @ cov_inv @ jac

def array_delta(name: str, num: np.ndarray, ref: np.ndarray) -> None:
    """Computes and prints the difference between two arrays.

    Args:
        name: Name of the array (for printing).
        num: Numeric array.
        ref: Reference (analytic) array.

    Returns:
        None
    """
    d = num - ref
    print(f"\n{name} (numeric):\n{num}")
    print(f"\n{name} (analytic):\n{ref}")
    print(f"\n $\\Delta$ = numeric - analytic:\n{d}")
    print(f"$\\max |\\Delta|$ = {np.max(np.abs(d)):.3e}, $|| \\Delta||_2$  = {np.linalg.norm(d):.3e}")

In [ ]:
def plot_fisher_ellipse(theta0: np.ndarray,
                        fisher_matrix: np.ndarray,
                        *,
                        level: float = PCT_2D_1SIG,
                        ax=None,
                        label: str | None = None,
                        lw: float = 2.0,
                        ls: str = "-"):
    """Plots the Fisher ellipse for a 2×2 Fisher matrix `F2` at parameter point `theta0`.

    Args:
        theta0: Center of the ellipse (2D parameter point).
        fisher_matrix: Fisher matrix (shape 2x2).
        level: Confidence level for the ellipse (0.393, 0.683, 0.955, 0.997).
        ax: Matplotlib Axes to plot on (creates new if None).
        label: Label for the ellipse (for legend).
        lw: Line width for the ellipse.
        ls: Line style for the ellipse.

    Returns:
        Matplotlib Axes with the ellipse plotted.
    """
    f_mat = np.asarray(fisher_matrix, dtype=float)
    if f_mat.shape != (2, 2):
        raise ValueError(f"Expected a 2×2 Fisher; got shape {f_mat.shape}.")
    if not np.allclose(f_mat, f_mat.T, rtol=1e-10, atol=1e-12):
        f_mat = 0.5 * (f_mat + f_mat.T)  # symmetrize

    # 2 dof chi2 quantiles mapping
    level_to_k2 = {0.393: 1.00, 0.683: 2.30, 0.955: 6.17, 0.997: 11.8}
    k2 = level_to_k2.get(level, 2.30)

    param_cov = np.linalg.pinv(f_mat, rcond=1e-12)  # covariance
    vals, vecs = np.linalg.eigh(param_cov)  # ascending eigenvalues
    t = np.linspace(0, 2*np.pi, 400)
    circle = np.vstack([np.cos(t), np.sin(t)])
    ellipse = vecs @ np.diag(np.sqrt(vals * k2)) @ circle
    blue_color = "#3b9ab2"

    if ax is None:
        _, ax = plt.subplots()
    ax.plot(theta0[0] + ellipse[0], theta0[1] + ellipse[1],
            label=label, linewidth=lw, linestyle=ls, color=blue_color)
    ax.scatter([theta0[0]], [theta0[1]], s=25, color=blue_color)
    ax.set_xlabel(r"parameter $\theta_1$", fontsize=13)
    ax.set_ylabel(r"parameter $\theta_2$", fontsize=13)
    ax.set_aspect("equal", adjustable="box")
    if label:
        ax.legend(frameon=False, fontsize=13)
    return ax

In [ ]:
# Compute Fisher numerically (ForecastKit) vs analytic, then print 1σ and 2σ marginals
method = "adaptive"   # change to "finite" if desired
theta0 = np.array([1.0, 2.0], dtype=float)
cov = np.array([[1.0, 0.0], [0.0, 1.0]], dtype=float)  # identity covariance on observables

print("=== Fisher demo ===")
print("fiducial $\\theta_0$ =", theta0)
print("covariance $C$ =\n", cov)
print("backend method =", method)

# ForecastKit Fisher
fk = ForecastKit(function=model, theta0=theta0, cov=cov)
fish_num = fk.fisher(method=method)
fish_ref = fisher_analytic(theta0[0], theta0[1], cov)
array_delta("Fisher", fish_num, fish_ref)

# Parameter covariance and marginal errors
fish_inv = np.linalg.pinv(fish_num, rcond=1e-12)  # using pseudo-inverse for stability
sigma_1 = np.sqrt(np.diag(fish_inv))
sigma_2 = 2.0 * sigma_1
print("\nParameter covariance (F^{-1}):\n", fish_inv)
print(f"\n1σ ({PCT_1D_1SIG*100:.2f}%) errors on (θ1, θ2):", sigma_1)
print(f"2σ ({PCT_1D_2SIG*100:.2f}%) errors on (θ1, θ2):", sigma_2)

In [ ]:
# Plot 1sigma (68.3%) and 2sigma (95.5%) ellipses
fig, ax = plt.subplots()
plot_fisher_ellipse(theta0, fish_num, level=PCT_2D_1SIG, ax=ax, label="1σ (68.3%)", ls="--", lw=2.0)
plot_fisher_ellipse(theta0, fish_num, level=PCT_2D_2SIG, ax=ax, label="2σ (95.5%)", ls="-",  lw=2.0)
ax.set_title("Fisher 1σ (68.3%) and 2σ (95.5%) ellipses")
plt.tight_layout()
plt.show()

print("Done. Another ellipse, another insight.")